In [ ]:
#Create initial database from csv
import csv, sqlite3

con = sqlite3.connect("db/db_tweets.db")
cur = con.cursor()
cur.execute("CREATE TABLE tweets (party, handle, tweet);")

with open('original_data/ExtractedTweets.csv','rt',encoding='utf-8') as fin:
    dr = csv.DictReader(fin)
    to_db = [(i['Party'], i['Handle'], i['Tweet']) for i in dr]

cur.executemany("INSERT INTO tweets (party, handle, tweet) VALUES (?, ?, ?);", to_db)
con.commit()
con.close()

In [ ]:
# Print tweets from db
import sqlite3

conn = sqlite3.connect('db/db_tweets.db')
cur = conn.cursor()

def get_posts(x):
    cur.execute("SELECT * FROM tweets")
    return cur.fetchmany(x)
    
def print_tweet(t):
    print('\n@{} [{}]\n-----\n{}\n'.format(t[1], t[0], t[2]))

for tweet in get_posts(10):
    print_tweet(tweet)

conn.close()

In [ ]:
# Print tweets from db
import sqlite3
import re

conn = sqlite3.connect('db/db_tweets.db')
cur = conn.cursor()

def analyze_mud():
#     print("-"*30)
#     print("DEMS")
#     print("-"*30)
    cur.execute("SELECT tweet FROM tweets WHERE party='Democrat'")
    mud_d = get_mud(cur.fetchall(), True)
#     print("\n"*100)
#     print("-"*30)
#     print("REPS")
#     print("-"*30)
    cur.execute("SELECT tweet FROM tweets WHERE party='Republican'")
    mud_r = get_mud(cur.fetchall(), False)
    print("\n\n---RESULTS---\n\nDEM MUD: {}\nREP MUD: {}".format(mud_d, mud_r))
    
def get_mud(tweets, dem):
    regex = r""
    result = 0
    if dem:
        regex = r"(?!\brepublic\b)\brepub+|\bgop\b|\bconservative+|\bright\w*wing\b"
    else:
        regex = r"(?!\bdemarest)(?!\bdemocracy\b)\bdem+|\bdnc\b|\bliberal+|\blibs+|\bleft\w*wing\b"
    for tweet in tweets:
        if(re.search(regex, tweet[0], re.I)):
            #print("\n{}\n".format(tweet[0]))
            result+=1
    return result

analyze_mud()

conn.close()

In [8]:
# Print tweets from db
import sqlite3
import re

conn = sqlite3.connect('db/db_tweets.db')
cur = conn.cursor()

def analyze_words(x):
    print("-"*30)
    print("DEMS")
    print("-"*30)
    cur.execute("SELECT tweet FROM tweets WHERE party='Democrat'")
    analyze(cur.fetchmany(x))
    print("\n"*3)
    print("-"*30)
    print("REPS")
    print("-"*30)
    cur.execute("SELECT tweet FROM tweets WHERE party='Republican'")
    analyze(cur.fetchmany(x))
    
def filter_word(word):
    wordFilterList = ['the', 'a', 'an', 'is', 'and', 'to', 'for', 'that', 'of', 'in', 'with']
    if not word.lower() in wordFilterList:
        return True
    
def analyze(tweets):
#     wordlist = []
#     for tweet in tweets:
#         wordlist.extend(tweet[0].split())

#     wordfreq = []
#     for w in wordlist:
#         wordfreq.append(wordlist.count(w))

#     print("List\n" + str(wordlist) + "\n")
#     print("Frequencies\n" + str(wordfreq) + "\n")
#     print("Word Frequencies\n" + str(list(zip(wordlist, wordfreq))))

    dWords = dict()
    for tweet in tweets:
        for word in tweet[0].split():
            if not filter_word(word):
                continue
            elif word in dWords:
                dWords[word] += 1
            else:
                dWords[word] = 1
    
    d_view = [ (v,k) for k,v in dWords.items() ]
    d_view.sort(reverse=True) # natively sort tuples by first element
    
    index = 0
    for v,k in d_view:
        index+=1
        if index > 10:
            break
        print("{} - {}: {}".format(index,k,v))
#     print(dWords)

analyze_words(500)

conn.close()

------------------------------
DEMS
------------------------------
1 - RT: 170
2 - our: 73
3 - on: 71
4 - @RepDarrenSoto: 65
5 - at: 62
6 - &amp;: 57
7 - you: 56
8 - this: 50
9 - I: 42
10 - my: 39




------------------------------
REPS
------------------------------
1 - our: 124
2 - RT: 85
3 - on: 82
4 - we: 60
5 - are: 59
6 - #VA05: 55
7 - I: 53
8 - at: 45
9 - you: 42
10 - this: 40
